# Function to set up text alert messages

This is a scratch notebook to develop a script for messaging people about a spike at a sensor near them

## Import Packages

In [1]:
### Import Packages

# File Manipulation

import os # For working with Operating System
import sys # System arguments
from dotenv import load_dotenv # Loading .env info

# Web

import requests # Accessing the Web

# Time

import datetime as dt # Working with dates/times
import pytz # Timezones

# Database 

import psycopg2
from psycopg2 import sql

# Data Manipulation

import numpy as np
import geopandas as gpd
import pandas as pd

In [2]:
## Database Credentials

load_dotenv()

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))    

# Current

## New Alert

### Short Version

In [5]:
# Shorter version (1 segment) - TWILIO charges by segment (160 characters)

sensor_index = 145604

message = f'''SPIKE ALERT!
Air quality is unhealthy in your area
https://map.purpleair.com/?select={sensor_index}/44.9723/-93.2447

Reply STOP to end alerts'''

In [6]:
print(message)
print('\n\n MESSAGE LENGTH:', len(message), 'Characters')

SPIKE ALERT!
Air quality is unhealthy in your area
https://map.purpleair.com/?select=145604/44.9723/-93.2447

Reply STOP to end alerts


 MESSAGE LENGTH: 134 Characters


## End Alert

In [7]:
# Dummy Variables

duration = 60
max_reading = 50.7
report_id = 'XXXXX-102623'

base_report_url = 'https://redcap.ahc.umn.edu/surveys/?s=LN3HHDCJXYCKFCLE'

In [8]:
message = f'''Alert Over
Duration: {duration} minutes 
Max value: {max_reading} ug/m3

Report here - {base_report_url+ '&report_id=' + report_id}'''

# See https://help.redcap.ualberta.ca/help-and-faq/survey-parameters for filling in variable in url

In [9]:
print(message)
print('\n\n MESSAGE LENGTH:', len(message), 'Characters') # TWILIO charges by segment (160 characters)

Alert Over
Duration: 60 minutes 
Max value: 50.7 ug/m3

Report here - https://redcap.ahc.umn.edu/surveys/?s=LN3HHDCJXYCKFCLE&report_id=XXXXX-102623


 MESSAGE LENGTH: 147 Characters


# ARCHIVE

### New Alert - Long Version

Get direction and distance from intersection they signed up for
Too long and costly a query, we will go with a simpler message seen 4 cells down

In [3]:
# Dummy variables

sensor_indices = [145604, 145604]
readings = [32.7, 32.7]
intersection_indices = [203, 315]


In [ ]:
# Using PostGIS

conn = psycopg2.connect(**pg_connection_dict)

# Create json cursor
cur = conn.cursor()

# Get the example as a Geojson
cmd = sql.SQL('''
WITH sensor_data as
(
SELECT * 
FROM "PurpleAir Stations" p
JOIN unnest({}::int[])
WITH ORDINALITY t(sensor_index, ord)
USING (sensor_index)
ORDER BY t.ord
),
intersection_data as
(
SELECT *
FROM "Road Intersections" i
JOIN unnest({}::int[])
WITH ORDINALITY t(intersection_index, ord)
USING (intersection_index)
ORDER BY t.ord
)
SELECT degrees(ST_Azimuth(ST_Transform(i.geometry, 26915), ST_Transform(p.geometry,26915))), -- https://gis.stackexchange.com/questions/54427/how-to-find-out-direction-postgis
ST_Distance(ST_Transform(i.geometry, 26915), ST_Transform(p.geometry,26915)),
i."NS_cross_street",
i."EW_cross_street", i.ord
FROM sensor_data p
INNER JOIN intersection_data i ON (p.ord = i.ord);
''').format(sql.Literal(sensor_indices), sql.Literal(intersection_indices))

cur.execute(cmd) # Execute

conn.commit() # Committ command

response = cur.fetchall()

In [6]:
response

[(313.3178047527333, 1164.482361384622, '31st Ave S', 'E 26th St', 1),
 (270.90272804702454, 246.6969441842309, '26th Ave S', 'E Franklin Ave', 2)]

In [7]:
# Will return some values that we will unpack...

degree, distance, ns_cross_street, ew_cross_street, order = response[0]

In [8]:
def degrees_to_cardinal(degree):
    '''
    A Function thtat gets the cardinal direction from a degree (azimuth)
    note: this is highly approximate...
    from https://gist.github.com/RobertSudwarts/acf8df23a16afdb5837f
    '''
    dirs = np.array(["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE",
                     "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"], dtype='U')
    ix = int(np.round(degree / (360. / len(dirs)))) % 16
    return dirs[ix]

In [9]:
direction_string = degrees_to_cardinal(degree)

In [10]:
# Here is the url to see a sensor:

# https://map.purpleair.com/?select=sensor_index/44.9723/-93.2447

url = 'https://map.purpleair.com/?select=143916/44.9723/-93.2447'

print(url)

https://map.purpleair.com/?select=143916/44.9723/-93.2447


In [ ]:
# Long Version (2 segments)

message = f'''SPIKE ALERT! 
PurpleAir Sensor {sensor_indices[order-1]} is reading {readings[order-1]} micrograms/meter^3.
This sensor is about {round(distance/1609,2)} miles {direction_string} from the intersection of {ew_cross_street} and {ns_cross_street}. 
Webmap - https://map.purpleair.com/?select={sensor_indices[order-1]}/44.9723/-93.2447

Please reply with STOP to be removed from SpikeAlerts.'''

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def new_alert_messages(sensor_indices, readings, intersection_indices, pg_connection_dict):
    '''
    Get a list of messages for new alerts
    # all inputs should be lists!
    # Composes the messages and return as a list of messages
    '''
    
    ## Query for relevant information
    
    conn = psycopg2.connect(**pg_connection_dict)

    # Create json cursor
    cur = conn.cursor()

    # Get direction and distance from intersection to the sensors using POSTGIS
    cmd = sql.SQL('''
    WITH sensor_data as
    (
    SELECT * 
    FROM "PurpleAir Stations" p
    JOIN unnest({}::int[])
    WITH ORDINALITY t(sensor_index, ord)
    USING (sensor_index)
    ORDER BY t.ord
    ),
    intersection_data as
    (
    SELECT *
    FROM "Road Intersections" i
    JOIN unnest({}::int[])
    WITH ORDINALITY t(intersection_index, ord)
    USING (intersection_index)
    ORDER BY t.ord
    )
    SELECT degrees(ST_Azimuth(ST_Transform(i.geometry, 26915), ST_Transform(p.geometry,26915))), -- https://gis.stackexchange.com/questions/54427/how-to-find-out-direction-postgis
    ST_Distance(ST_Transform(i.geometry, 26915), ST_Transform(p.geometry,26915)),
    i."NS_cross_street",
    i."EW_cross_street", i.ord
    FROM sensor_data p
    INNER JOIN intersection_data i ON (p.ord = i.ord);
    ''').format(sql.Literal(sensor_indices), sql.Literal(intersection_indices))

    cur.execute(cmd) # Execute

    conn.commit() # Committ command

    response = cur.fetchall()
    
    # Now compose the messages
    
    messages = []
    
    for r in response:
    
        degree, distance, ns_cross_street, ew_cross_street, order = r # Unpack the tuple
        
        direction_string = degrees_to_cardinal(degree)
        
        # Long Version (2 segments)

        message = f'''SPIKE ALERT! 
PurpleAir Sensor {sensor_indices[order-1]} is reading {readings[order-1]} micrograms/meter^3.
This sensor is about {round(distance/1609,2)} miles {direction_string} from the intersection of {ew_cross_street} and {ns_cross_street}. 
Webmap - https://map.purpleair.com/?select={sensor_indices[order-1]}/44.9723/-93.2447

Please reply with STOP to be removed from SpikeAlerts.'''
        
        messages += [message]
        
    return messages